In [14]:
pip install pandas transformers


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import csv
import re
import string
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

df_ytb = pd.read_csv("/content/drive/MyDrive/CSV FILES/ytb_Perks Depression Scene.csv")

# Remove column name 'Unnamed: 0'
dfc = df_ytb.drop(['Unnamed: 0','username'], axis=1)
# dfc = df_ytb.drop(index=11)
dfc = dfc.reset_index(drop=True)

#list_of_sentences = dfc['comment'].tolist()
#dfc
def clean_html(text):
    clean=re.compile('<.*?>')
    cleantext=re.sub(clean,'',text)
    return cleantext

# first round of cleaning
def clean_text1(text):
    text=text.lower()
    text=re.sub('','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub('\w*\d\w*','',text)
    return text

# second round of cleaning
def clean_text2(text):
    text=re.sub('[''"",,,]','',text)
    text=re.sub('\n','',text)
    return text

cleaned_html=lambda x:clean_html(x)
cleaned1=lambda x:clean_text1(x)
cleaned2=lambda x:clean_text2(x)

dfc['review'] = dfc['comment']
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned_html))
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned1))
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned2))
print(dfc.shape)
dfc.head(10)

first10comments = dfc.review.values

sentences = first10comments
list_of_sentences = dfc['comment'].tolist()


(100, 2)


In [17]:
tokenizer = AutoTokenizer.from_pretrained("aiknowyou/it-emotion-analyzer")
model = AutoModelForSequenceClassification.from_pretrained("aiknowyou/it-emotion-analyzer")

emotion_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
emotion_analysis(list_of_sentences)
results = emotion_analysis(list_of_sentences)
results

[{'label': '2', 'score': 0.8129293322563171},
 {'label': '2', 'score': 0.7877767086029053},
 {'label': '1', 'score': 0.6275376677513123},
 {'label': '2', 'score': 0.898472785949707},
 {'label': '2', 'score': 0.48179036378860474},
 {'label': '0', 'score': 0.6376990675926208},
 {'label': '2', 'score': 0.6590078473091125},
 {'label': '0', 'score': 0.635502815246582},
 {'label': '2', 'score': 0.8972740769386292},
 {'label': '2', 'score': 0.8043369054794312},
 {'label': '2', 'score': 0.5642092823982239},
 {'label': '2', 'score': 0.6100863814353943},
 {'label': '2', 'score': 0.9294015169143677},
 {'label': '1', 'score': 0.5519399046897888},
 {'label': '2', 'score': 0.5606153011322021},
 {'label': '3', 'score': 0.3730330169200897},
 {'label': '2', 'score': 0.7581339478492737},
 {'label': '0', 'score': 0.7969208359718323},
 {'label': '2', 'score': 0.8313855528831482},
 {'label': '1', 'score': 0.5517070293426514},
 {'label': '1', 'score': 0.6274769902229309},
 {'label': '2', 'score': 0.82825958

In [18]:
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
# Map numerical labels to emotions in the results
dfc['Emotion'] = [emotion_mapping.get(result['label'], result['label']) for result in results]
dfc['Score'] = [result['score'] for result in results]

final =dfc[['comment', 'Emotion', 'Score']]
final


,comment,Emotion,Score
0,I related so much to this scene when I saw thi...,2,0.812929
1,So deep😢😢. Yet so sad.,2,0.787777
2,Incredible actor,1,0.627538
3,My eyes watered up the second his sister could...,2,0.898473
4,"Reading the book, the way Charlie behaves, com...",2,0.481790
...,...,...,...
95,This scene captures so much of my life.,2,0.467097
96,Does he kind of remind any of you of Donnie Da...,2,0.674804
97,I go through depression as well,0,0.888859
98,The accuracy,1,0.534731


In [19]:
# Create a new column 'mapped_emotion' using emotion_mapping
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')

final['Mapped_Emotion'] = final['Emotion'].map(emotion_mapping)

# Display the updated 'final' DataFrame with the new column
print(final[['Emotion', 'Mapped_Emotion']])

    Emotion Mapped_Emotion
0         2           love
1         2           love
2         1            joy
3         2           love
4         2           love
..      ...            ...
95        2           love
96        2           love
97        0        sadness
98        1            joy
99        2           love

[100 rows x 2 columns]


<ipython-input-19-a9bc13b09b5c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')


In [20]:
# Add sentiment analysis results to the DataFrame
# Save the DataFrame with sentiment results to a new CSV file
output_csv_path = "/content/drive/MyDrive/ytb_Perks Depression Scene_with_sentiments.csv"
final.to_csv(output_csv_path, index=False)

# Display the first few rows of the DataFrame with sentiment results
final

,comment,Emotion,Score,Mapped_Emotion
0,I related so much to this scene when I saw thi...,2,0.812929,love
1,So deep😢😢. Yet so sad.,2,0.787777,love
2,Incredible actor,1,0.627538,joy
3,My eyes watered up the second his sister could...,2,0.898473,love
4,"Reading the book, the way Charlie behaves, com...",2,0.481790,love
...,...,...,...,...
95,This scene captures so much of my life.,2,0.467097,love
96,Does he kind of remind any of you of Donnie Da...,2,0.674804,love
97,I go through depression as well,0,0.888859,sadness
98,The accuracy,1,0.534731,joy
